In [ ]:
#restart kernel
import os
os._exit(00)

In [3]:
import sys
import argparse
import pygame
from nodeProp import Node 
import random 
import numpy as np
import simulation as sm
import state as st
import heuristics
import graphics as gp
import node_manipulator as nd
import nodeProp as np
import properties as pr

pygame 2.0.0.dev6 (SDL 2.0.10, python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
from cpmp import Greedy
from cpmp import Layout
import copy

layout0 = Layout.read_file("cpmp/Instancias/BF//BF32/cpmp_20_8_128_52_96_8.bay", 8)

def simulation(p_key):

    path = [] 
    
    # Se crea una copia auxiliar que guarde la  informacion del nodo actual
    aux = sm.StateMap[p_key] 
    path.insert(0,len(sm.StateMap[p_key].ChildList)+1)
    
    while aux.IdLastChild != None: 
    
        # Se mete a la pila el nuevo hijo
        path.insert(0,aux.IdLastChild) 
        
        # Se obtiene el padre de nodo (con toda su informacion para crear el camino hacia arriba) 
        aux = aux.Parent 
        # Se vuelve a iterar siempre y cuando su IdChild no None, que es el caso del nodo raiz
        
    #simulation
    layout = copy.deepcopy(layout0)
     
    eval, actions = Greedy.simulation(layout, path)
    return (140-eval)/40, actions


In [5]:
def click_node(NOS,id,n_manipulator):
    #graphic staff
    for i in range(0,NOS):
        if i==0: id_child = n_manipulator.generate_son(id)
        else: n_manipulator.generate_son(id)

    for i in range(NOS):
        eval, actions = simulation(id)
        sm.CreateState(id_child, id, actions, eval)
        id_child += 1
        
        
    #more graphic staff
    for node in n_manipulator.nodes:
        newColor = pr.StateColor(node.id, len(n_manipulator.nodes)) 
        node.color_to(newColor)

In [6]:
raiz = Node([27, 27], [200, 200, 200], 1, 0) 
n_manipulator = nd.NodeManipulator(raiz) 

pygame.init()
screen = pygame.display.set_mode((900, 500), pygame.SRCALPHA, 32)
pygame.display.set_caption("Node Plotter")

#click_node(2,0,n_manipulator)
#click_node(2,0,n_manipulator)
#click_node(2,1,n_manipulator)

In [ ]:
#visualización (fase recopilación de información)
done=False
while not done:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            done=True
        elif event.type == pygame.MOUSEBUTTONDOWN:
            x, y = event.pos
            id = n_manipulator.get_node_id(x,y)

            #ningun nodo seleccionado (seleccion automatica)
            if id == -1 or len(sm.StateMap[id].ChildList) >= sm.StateMap[id].NumActions : 
                continue
            
            # En etapa 1 (recuperación de información)
            # -->  ALMACENAR INFORMACION RELACIONADA A LA DECISION DEL USUARIO
            
            simulation(id)
            click_node(2,id,n_manipulator)
            
    if not done:
        n_manipulator.update()

        screen.fill((33, 33, 33))
        n_manipulator.draw(screen)
        pygame.display.update()
        pygame.time.wait(10)
        